# レッスン 06 LangChain

LangChainは、2022年末に大規模言語モデル技術が爆発的に普及して以来、初めての本格的な大規模言語モデル開発フレームワークと言えるでしょう。大規模言語モデルは本質的に実際の問題を直接解決することはできず、単にテキストを分析、推論、生成できるブラックボックスに過ぎません。今日に至るまで、すべての開発者は大規模言語モデルの強力な能力を実際のアプリケーションシナリオと組み合わせる方法を模索し続けています。そのような中、LangChainの登場により、大規模言語モデルの開発が一気に簡単になりました。LangChainは、大規模言語モデル開発プロセスでよく使用される機能、ツール、フローなどをすべてコンポーネントとしてカプセル化し、開発者がレゴブロックを組み立てるように、さまざまなシナリオのニーズに適した大規模言語モデルアプリケーションを迅速に構築できるようにしました。

LangChainの最初のバージョンは2022年10月にオープンソース化され、現在でも非常に速いペースで反復的にアップグレードされ続けています。オープンソースのPython/TSフレームワークから徐々に発展し、「チェーン」や「エージェント」などのコアコンポーネントを形成し、現在ではエンタープライズレベルの段階に進み、LangChain AIへと発展しました。これは現在、エージェント技術分野で最大のオープンソースエコシステムを持ち、複数のオープンソースプロジェクトフレームワークを派生させ、それぞれが大規模言語モデルの技術分野において異なる開発タスクの役割を担っています。

比喩的に言えば、LangChainの機能的な位置づけは、実際には現在私たちが呼んでいるエージェント開発フレームワークではなく、大規模言語モデルの機能拡張ツールです。LangChainを活用すれば、GPT-3モデルの時代であっても、モデルに対話、記憶の保持、さらには構造化された出力などの機能を実現させることができます。

## LangChainインストール

`LangChain`を使用して大規模言語モデルアプリケーションを開発する場合、`LangChain`の依存パッケージをインストールする必要があります。インストールコマンドは以下の通りです:

LangChain >= 1.0.0、Python >= 3.11であることを確認してください。バージョンが低い場合は、`pip install --upgrade <パッケージ名>`を使用してアップグレードしてください。

In [2]:
!pip install langchain

  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 28.2 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.13.2
    Uninstalling typing_extensions-4.13.2:
      Successfully uninstalled typing_extensions-4.13.2


In [4]:
import langchain

print(langchain.__version__)

1.1.0


## Langchain は API を介してオンラインLLMに接続します

`LangChain`の開発を始める前に、まず呼び出し可能な大規模言語モデルを準備する必要があります。ここでは`DeepSeek`の大規模言語モデルを使用し、`DeepSeek`公式の`API_KEY`を使って呼び出しを行います(ここではすでにコード内に提供されています)。自分て使用する場合は、まず`DeepSeek`公式サイトで登録を行い、新しい`API_Key`を作成する必要があります。公式サイトのアドレスは以下の通りです: https://platform.deepseek.com/usage

In [ ]:
!pip install openai